## Estrategias de clasificación

Casi todo en DynamoDB fluye a través de las claves primarias. Esto se aplica a las relaciones, al filtrado y a la ordenación. Al igual que con las uniones y el filtrado, necesitas organizar tus elementos para que estén ordenados de antemano.

Si necesita un orden específico al recuperar varios elementos en DynamoDB, debe seguir dos reglas principales. En primer lugar, debe utilizar una clave primaria compuesta. En segundo lugar, todo el ordenamiento debe realizarse con la clave de ordenación de una colección de elementos concreta.

### Ordenación lexicográfica

Una versión simplificada de la ordenación en bytes UTF-8 es decir que la ordenación es lexicográfica. Este orden es básicamente el orden del diccionario con dos salvedades:
* Todas las mayúsculas van antes que las minúsculas
* Los números y símbolos (por ejemplo, # o $) también son relevantes.

Donde más tropiezos veo con el orden lexicográfico es en el olvido de la regla de las mayúsculas. Para evitar comportamientos extraños en torno a esto, se deben estandarizar las claves de ordenación en mayúsculas o minúsculas:



In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime
from pprint import pprint

In [2]:
dt=DynamoTable()
try:
    dt.select_table('SampleTable')
    print(dt)
except:
    dt.create_table(
        table_name='SampleTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
    )

- Table name: SampleTable            
- Table arn: arn:aws:dynamodb:us-east-1:466206880806:table/SampleTable            
- Table creation: 2023-07-04 17:52:45            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [5]:
names = {'DeBrie': 'Alex DeBrie', 'Dean': 'James Dean', 'Dern': 'Laura Dern'}
for key, value in names.items():
    response = dt.table.put_item(
        Item={
            'PK': 'D',
            'SK': key,
            'Name': value
        }
    )

In [7]:
response = dt.query(pk_value='D')
print(response['Items'])

[{'SK': 'DeBrie', 'PK': 'D', 'Name': 'Alex DeBrie'}, {'SK': 'Dean', 'PK': 'D', 'Name': 'James Dean'}, {'SK': 'Dern', 'PK': 'D', 'Name': 'Laura Dern'}]


Te sorprenderá ver que DeBrie va antes que Dean. Esto se debe a que las mayúsculas preceden a las minúsculas.

In [8]:
names = {'DEBRIE': 'Alex DeBrie', 'DEAN': 'James Dean', 'DERN': 'Laura Dern'}
for key, value in names.items():
    response = dt.table.put_item(
        Item={
            'PK': 'Dnew',
            'SK': key,
            'Name': value
        }
    )

In [9]:
response = dt.query(pk_value='Dnew')
print(response['Items'])

[{'SK': 'DEAN', 'PK': 'Dnew', 'Name': 'James Dean'}, {'SK': 'DEBRIE', 'PK': 'Dnew', 'Name': 'Alex DeBrie'}, {'SK': 'DERN', 'PK': 'Dnew', 'Name': 'Laura Dern'}]


Con todos los apellidos en mayúsculas, ahora están ordenados como cabría esperar.

### ID únicos y ordenables

Una necesidad común es tener IDs únicos y ordenables.  Esto ocurre cuando se necesita un identificador único para un elemento (e idealmente un mecanismo que sea amigable con la URL) pero también se quiere poder ordenar un grupo de estos elementos cronológicamente.

Hay algunas opciones en este ámbito, pero yo prefiero la implementación de KSUID de la gente de Segment. Un KSUID es un un identificador único al que se antepone una marca de tiempo, pero que también contiene suficiente aleatoriedad para que las colisiones sean muy improbables. En total, se obtiene una cadena de 27 caracteres que es más única que un UUIDv4, al tiempo que conserva la ordenación lexicográfica.

**¿Por qué utilizar un KSUID?**

* Ordenable por fecha y hora
* 128 bits de datos aleatorios
* Representaciones portables y clasificables por lexografía

In [18]:
#!pip install svix-ksuid
from ksuid import Ksuid
from ksuid import KsuidMs

ksuid = Ksuid()
#ksuid = KsuidMs()

In [21]:
print(f"Base62: {ksuid}")
print(f"Datetime: {ksuid.datetime}")
print(f"Timestamp: {ksuid.timestamp}")
print(f"Payload: {ksuid.payload}")

Base62: 2S7lFS9cAv7q20FnKAJvzaHiUhl
Datetime: 2023-07-04 21:16:35+00:00
Timestamp: 1688505395.0
Payload: b'\xe8+L\x97/\xff\x0b\x89\x96\xe3SkK\x15\xbe\x01'


In [36]:
text = '1YnlHOfSSk3DhX4BR6lMAceAo1V'
ksuid_1 = Ksuid.from_base62(data=text)

In [39]:
print(f'Datetime: {ksuid_1.datetime}')
print(f'Timestamp: {ksuid_1.timestamp}')

Datetime: 2020-03-07 13:02:30+00:00
Timestamp: 1583586150.0


In [49]:
dt=DynamoTable()
try:
    dt.select_table('newSampleTable')
    print(dt)
except:
    dt.create_table(
        table_name='newSampleTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='B'
    )

Table created successfully!


In [61]:
orders = ['Product1', 'Product2', 'Product3', 'Product4', 'Product5']
for order in orders:
    ksuid = Ksuid()
    response = dt.table.put_item(
        Item={
            'PK': 'ORDER#220',
            'SK': 'ORDERID#' + str(ksuid),
            'Product': order
        }
    )

In [62]:
response = dt.query(pk_value='ORDER#220')
pprint(response['Items'])

[{'PK': 'ORDER#220',
  'Product': 'Product1',
  'SK': 'ORDERID#2S7pOaGwsooZHwQF9Cjb73DksHd'},
 {'PK': 'ORDER#220',
  'Product': 'Product2',
  'SK': 'ORDERID#2S7pOiNN24LzMKTLwGdXJuWehZ2'},
 {'PK': 'ORDER#220',
  'Product': 'Product3',
  'SK': 'ORDERID#2S7pOluIatWmAJkSExVUTtApxnA'},
 {'PK': 'ORDER#220',
  'Product': 'Product4',
  'SK': 'ORDERID#2S7pOuwiXwYHEMoUOS0FPyZH6CZ'},
 {'PK': 'ORDER#220',
  'Product': 'Product5',
  'SK': 'ORDERID#2S7pOvLtZakgY7sAT9AmCUyzBRv'}]


### Ascendente frente a descendente

Se puede utilizar la propiedad `ScanIndexForward` para indicar a DynamoDB cómo ordenar los elementos. Por defecto, DynamoDB leerá los elementos en orden ascendente. Si trabaja con palabras, esto significa que empezará por *aardvark* y seguirá por *zebra*.

Si está trabajando con marcas de tiempo, esto significa comenzar en el año 1900 y avanzar hacia el año 2020.
Puede invertir esto utilizando ScanIndexForward=False, lo que significa que leerá los elementos en orden descendente. Esto es útil para varias ocasiones, como cuando quieres obtener las marcas de tiempo más recientes o quieres encontrar las puntuaciones más altas en la tabla de clasificación.

Por ejemplo, imagine que tiene un dispositivo IoT que envía lecturas ocasionales de sensores. Uno de los patrones de acceso habituales es obtener el elemento Dispositivo y las 5 lecturas más recientes del dispositivo.

In [8]:
dt.table.put_item(
    Item={
        'PK': 'DEVICE#123',
        'SK': 'DEVICE#123',
        'DeviceLocation': 'Omaha, NE'
    }
)

dates = ['2020-03-14T10:33:00', '2020-03-14T10:34:00', '2020-03-14T10:35:00', '2020-03-14T10:36:00', '2020-03-14T10:37:00']
temp = [Decimal('72.5'), Decimal('72.6'), Decimal('72.7'), Decimal('72.8'), Decimal('72.9')]
for i in range(len(dates)):
    response = dt.table.put_item(
        Item={
            'PK': 'DEVICE#123',
            'SK': 'READING#' + dates[i],
            'Temperature': temp[i]
        }
    )

In [65]:
response = dt.query(pk_value='DEVICE#123')
pprint(response['Items'])

[{'DeviceLocation': 'Omaha, NE', 'PK': 'DEVICE#123', 'SK': 'DEVICE#123'},
 {'PK': 'DEVICE#123', 'SK': 'READING#2020-03-14T10:33:00', 'Temperature': 72.5},
 {'PK': 'DEVICE#123', 'SK': 'READING#2020-03-14T10:34:00', 'Temperature': 72.6},
 {'PK': 'DEVICE#123', 'SK': 'READING#2020-03-14T10:35:00', 'Temperature': 72.7},
 {'PK': 'DEVICE#123', 'SK': 'READING#2020-03-14T10:36:00', 'Temperature': 72.8},
 {'PK': 'DEVICE#123', 'SK': 'READING#2020-03-14T10:37:00', 'Temperature': 72.9}]


Observe que el elemento principal Dispositivo se encuentra antes que cualquiera de los elementos Lectura porque "DEVICE" está antes que "READING" en el alfabeto. Debido a esto, nuestra consulta para obtener el Dispositivo y las Lecturas recuperaría los elementos más antiguos. Si nuestra colección de ítems fuera grande, podríamos necesitar hacer múltiples peticiones de paginación para obtener los ítems más recientes.

Para evitar esto, podemos añadir un prefijo # a nuestros elementos de lectura(READING).

In [9]:
dt.table.put_item(
    Item={
        'PK': 'DEVICE#124',
        'SK': 'DEVICE#124',
        'DeviceLocation': 'New York, NY'
    }
)

for i in range(len(dates)):
    response = dt.table.put_item(
        Item={
            'PK': 'DEVICE#124',
            'SK': '#READING#' + dates[i],
            'Temperature': temp[i]
        }
    )

In [11]:
response = dt.query(pk_value='DEVICE#124')
pprint(response['Items'])

[{'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:33:00',
  'Temperature': 72.5},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:34:00',
  'Temperature': 72.6},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:35:00',
  'Temperature': 72.7},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:36:00',
  'Temperature': 72.8},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:37:00',
  'Temperature': 72.9},
 {'DeviceLocation': 'New York, NY', 'PK': 'DEVICE#124', 'SK': 'DEVICE#124'}]


Ahora podemos utilizar la API de consulta para obtener el elemento Dispositivo y los elementos de lectura más recientes empezando por el final de nuestra colección de elementos y utilizando la propiedad ScanIndexForward=False.

In [5]:
response = dt.query(pk_value='DEVICE#124', scan_index_forward=False)
pprint(response['Items'])

[{'DeviceLocation': 'New York, NY', 'PK': 'DEVICE#124', 'SK': 'DEVICE#124'},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:37:00',
  'Temperature': 72.9},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:36:00',
  'Temperature': 72.8},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:35:00',
  'Temperature': 72.7},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:34:00',
  'Temperature': 72.6},
 {'PK': 'DEVICE#124',
  'SK': '#READING#2020-03-14T10:33:00',
  'Temperature': 72.5}]
